In [1]:
import cv2

# Load the video
video_path = '006.mp4'
cap = cv2.VideoCapture(video_path)

# Get the total number of frames
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f"Number of frames: {frame_count}")

# Release the video capture object
cap.release()


Number of frames: 245


In [10]:
import cv2
import json
import os

# Ask user for paths
video_path = "./006.mp4"
annotation_path = "./006.mp4.json"

# Load Supervisely JSON
with open(annotation_path) as f:
    annotations = json.load(f)

# Create directories for frames and labels
os.makedirs('frames', exist_ok=True)
os.makedirs('labels', exist_ok=True)

# Extract video properties
cap = cv2.VideoCapture(video_path)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
height = annotations['size']['height']
width = annotations['size']['width']

# Function to convert to YOLO format
def convert_to_yolo(exterior, img_width, img_height):
    x_min, y_min = exterior[0]
    x_max, y_max = exterior[1]
    center_x = (x_min + x_max) / 2 / img_width
    center_y = (y_min + y_max) / 2 / img_height
    bbox_width = (x_max - x_min) / img_width
    bbox_height = (y_max - y_min) / img_height
    return center_x, center_y, bbox_width, bbox_height

# Initialize class_mapping if not already present
class_mapping = {}

# Check if class_mapping is empty; if not, use the existing mapping
if not class_mapping:
    for idx, obj in enumerate(annotations['objects']):
        if obj['classTitle'] not in class_mapping:
            class_mapping[obj['classTitle']] = idx  # Assign unique ID to each classTitle

# Iterate through frames and annotations
for frame in annotations['frames']:
    frame_index = frame['index']  # Get the correct frame index
    
    if frame_index < total_frames:  # Ensure index is within video length
        # Set video to the correct frame
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_index)
        ret, image = cap.read()
        
        if ret:
            # Save the current frame as an image
            frame_path = f"frames/frame_{frame_index:04d}.jpg"
            cv2.imwrite(frame_path, image)

            # Create corresponding label file
            label_path = f"labels/frame_{frame_index:04d}.txt"
            with open(label_path, 'w') as label_file:
                for figure in frame['figures']:
                    geometry = figure['geometry']['points']['exterior']
                    
                    # Convert bounding box to YOLO format
                    center_x, center_y, bbox_width, bbox_height = convert_to_yolo(geometry, width, height)
                    
                    # Get the class ID from the objectKey using class_mapping
                    object_key = figure['objectKey']
                    for obj in annotations['objects']:
                        if obj['key'] == object_key:
                            class_id = class_mapping[obj['classTitle']]
                            break
                    
                    # Write to label file with the correct class ID
                    label_file.write(f"{class_id} {center_x} {center_y} {bbox_width} {bbox_height}\n")

# Release the video capture
cap.release()


In [11]:
class_mapping

{'Whity': 0, 'greeny': 1, 'small greeny': 2, 'small whity': 3}

# Validate

In [ ]:
import cv2
import matplotlib.pyplot as plt
import os

def get_color(class_id):
    """Return the color for the bounding box based on the class ID."""
    colors = {
        0: (0, 0, 255),   # Red in BGR
        1: (0, 255, 0),   # Green in BGR
        2: (255, 0, 0),   # Blue in BGR
        3: (0, 255, 255)  # Yellow in BGR
    }
    return colors.get(class_id, (255, 255, 255))  # Default to white if class ID is unknown

def validate_yolo_annotations(yolo_file, image_file):
    # Load the image
    img = cv2.imread(image_file)
    if img is None:
        print(f"Error: Unable to load image {image_file}")
        return

    h, w, _ = img.shape

    # Read YOLO annotations
    with open(yolo_file, 'r') as file:
        lines = file.readlines()

    # Loop through each line in the YOLO file
    for line in lines:
        try:
            class_id, x_center, y_center, width, height = map(float, line.strip().split())
            # Convert YOLO format to bounding box coordinates
            x_min = int((x_center - width / 2) * w)
            x_max = int((x_center + width / 2) * w)
            y_min = int((y_center - height / 2) * h)
            y_max = int((y_center + height / 2) * h)

            # Get the color for the bounding box
            color = get_color(int(class_id))

            # Draw rectangle around the object
            cv2.rectangle(img, (x_min, y_min), (x_max, y_max), color, 2)
        except Exception as e:
            print(f"Error processing line '{line.strip()}': {e}")

    # Display the image using matplotlib
    plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
    plt.title(f'YOLO Annotations for {os.path.basename(image_file)}')
    plt.axis('off')
    plt.show()

def process_yolo_folder(labels_folder, images_folder):
    # Loop through the files in the images folder
    for image_filename in os.listdir(images_folder):
        if image_filename.lower().endswith(('.jpg', '.png', '.jpeg')):  # Supports jpg, png, jpeg
            image_file = os.path.join(images_folder, image_filename)

            # Assuming the label files have the same base name as the image files
            yolo_file = os.path.join(labels_folder, os.path.splitext(image_filename)[0] + '.txt')

            # Check if corresponding label file exists
            if os.path.exists(yolo_file):
                print(f'Processing {image_filename}')
                validate_yolo_annotations(yolo_file, image_file)
            else:
                print(f'No annotation file found for {image_filename}')

# Set the paths to your folders
labels_folder = './labels/'
images_folder = './frames/'

# Process all images in the folder
process_yolo_folder(labels_folder, images_folder)
